<a href="https://www.kaggle.com/code/milkbottledude/houseprices-machine-learning?scriptVersionId=207875803" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns')


/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# Data Configuration

In [2]:
# Quantifying BsmtQual, which represents the height of the basement
bsmtdict = {
    'Ex': 100,
    'Gd': 90,
    'TA': 80,
    'Fa': 70,
    'Po': 40,
    'NA': 0
}

# Quantifying HeatingQC: Heating quality and condition
heatQC = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
}

# label encoding the column 'MoSold'(represents the month the house was sold)
month = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}

# making a list of conditions that are in the variables condition1 and condition2
conditions = ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosN', 'PosA', 'RRNe', 'RRAe']
# Training Data
train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train_data['BsmtQual'] = train_data['BsmtQual'].map(bsmtdict)
train_data['HeatingQC'] = train_data['HeatingQC'].map(heatQC)
train_data['MoSold'] = train_data['MoSold'].map(month)

# combining the total porch area into the column 'TotalPorch'
train_data['TotalPorch'] = train_data['OpenPorchSF'] + train_data['EnclosedPorch'] + train_data['3SsnPorch'] + train_data['ScreenPorch'] 

# combining both condition1 and condition2 by manually making the dummy columns
for c in conditions:
    train_data[c] = False
for index, row in train_data.iterrows():
    c1 = row['Condition1']
    c2 = row['Condition2']
    for cond in [c1, c2]:
        train_data.at[index, cond] = True
train_data.drop(['Condition1', 'Condition2'], axis=1, inplace = True)
# dropping MiscFeatures
train_data.drop('MiscFeature', axis=1, inplace=True)
# getting dummy variables for categorical columns with qualitative data
X = pd.get_dummies(train_data.drop(['SalePrice', 'Id', 'YrSold'], axis=1))
y = train_data["SalePrice"]
# for i in X.columns:
#     print(i)
LFmean = X['LotFrontage'].mean()
X['GarageYrBlt'] = X['GarageYrBlt'].fillna(X['YearBuilt'])
X['MasVnrArea'] = X['MasVnrArea'].fillna(0)
X['LotFrontage'] = X['LotFrontage'].fillna(LFmean)
X['BsmtQual'] = X['BsmtQual'].fillna(X['BsmtQual'].median())


#maybe scale the year and squarearea of the garages, then create a new variable which is the product of both of them, that way no garage houses wld have values of 0

# Test(fr) Data
# all feature engineering done for training data will also be done to the test data below in similar fashion
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
TestIDs = test_data.Id
# quantitating bsmtqual
test_data['BsmtQual'] = test_data['BsmtQual'].map(bsmtdict)
test_data['MoSold'] = test_data['MoSold'].map(month)

# combining condition1 and condition2
for c in conditions:
    test_data[c] = False
for index, row in test_data.iterrows():
    c1 = row['Condition1']
    c2 = row['Condition2']
    for cond in [c1, c2]:
        test_data.at[index, cond] = True
test_data.drop(['Condition1', 'Condition2'], axis=1, inplace=True)

test_data.drop('MiscFeature', axis=1, inplace=True)

# test_data = test_data.drop(['OverallCond', 'OverallQual'], axis=1)
X_test_fr = pd.get_dummies(test_data)

X_test_fr['BsmtQual'] = X_test_fr['BsmtQual'].fillna(X_test_fr['BsmtQual'].median())


columns with Na values in train_data: 
1) LotFrontage (259 Na values) - represents the front yard area in front of a house's garage, gonna replace Na values with the mean as filling with 0 when the actual value is actually much higher and close to the mean value may rough up my model fitting.

2) MasVnrArea (8 NA values) - represents the area (in square feet) of the masonry veneer on the exterior of the house, can be an indicator of the quality or aesthetic value of a home’s exterior. Will replace Na values in this column with 0.

3) GarageYrBlt (81 Na values) - indicates the year in which the garage was constructed. If the house was built with an attached garage, this value would typically be the same as the year the house was built. Will replace Na values with the year the house was built.

In [3]:

#Checking for NaN values
# print(X_test_fr.isna().sum()[X_test_fr.isna().sum() > 0])
#Dealing with em
TestLFmean = X_test_fr['LotFrontage'].mean()
X_test_fr['GarageYrBlt'] = X_test_fr['GarageYrBlt'].fillna(X_test_fr['YearBuilt'])
X_test_fr['MasVnrArea'] = X_test_fr['MasVnrArea'].fillna(0)
X_test_fr['LotFrontage'] = X_test_fr['LotFrontage'].fillna(TestLFmean)
X_test_fr['GarageCars'] = X_test_fr['GarageCars'].fillna(1)
GAreaMean = X_test_fr['GarageArea'].mean()
X_test_fr['GarageArea'] = X_test_fr['GarageArea'].fillna(GAreaMean)
X_test_fr['BsmtUnfSF'] = X_test_fr['BsmtUnfSF'].fillna(X_test_fr['BsmtUnfSF'].mean())
X_test_fr['TotalBsmtSF'] = X_test_fr['TotalBsmtSF'].fillna(X_test_fr['TotalBsmtSF'].mean())
#Since the mode for BsmtFullBath. BsmtHalfBath, BsmtFinSF1, and BsmtFinSF2 are 0, ill just fillna with 0
X_test_fr['BsmtFullBath'] = X_test_fr['BsmtFullBath'].fillna(0)
X_test_fr['BsmtHalfBath'] = X_test_fr['BsmtHalfBath'].fillna(0)
X_test_fr['BsmtFinSF1'] = X_test_fr['BsmtFinSF1'].fillna(0)
X_test_fr['BsmtFinSF2'] = X_test_fr['BsmtFinSF2'].fillna(0)


for case in [X, X_test_fr]:
    #creating new column totalsf which is the total surface area of all the floors in the house
    case['TotalSF'] = case['1stFlrSF'] + case['2ndFlrSF']
    case.drop('1stFlrSF', axis = 1, inplace=True)
    case.drop('2ndFlrSF', axis = 1, inplace=True)
    #creating a new column TotalBth which is the total number of bathrms in the house
    case['TotalBth'] = case['BsmtFullBath'] + case['FullBath']
    case.drop('BsmtFullBath', axis = 1, inplace=True)
    # will be doing this for half bathrms as well, not just full bathrms
    case['TotalHalfBth'] = case['BsmtHalfBath'] + case['HalfBath']
    case.drop('BsmtHalfBath', axis = 1, inplace=True)
    # combining all the bathrooms tgthr (dividing no. of half bathrms by 2)
    case['TotalBth'] = case['TotalBth'] + case['TotalHalfBth'] * 0.5
    case.drop('TotalHalfBth', axis=1, inplace=True)
    #creating new columns Age, GAge, ReAge, which will basically be property age, and age of garage, and years since remodelling, respectively'
    case['Age'] = 2024 - case['YearBuilt']
    case['GAge'] = 2024 - case['GarageYrBlt']
    case['ReAge'] = 2024 - case['YearRemodAdd']
    case.drop('YearBuilt', axis = 1, inplace=True)
    case.drop('GarageYrBlt', axis = 1, inplace=True)
    case.drop('YearRemodAdd', axis = 1, inplace=True)
    case.drop('MiscVal', axis = 1, inplace=True)
    

print(X_test_fr[X_test_fr.isna().any(axis=1)])
print(X[X.isna().any(axis=1)])

Empty DataFrame
Columns: [Id, MSSubClass, LotFrontage, LotArea, OverallQual, OverallCond, MasVnrArea, BsmtQual, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, LowQualFinSF, GrLivArea, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, YrSold, Artery, Feedr, Norm, RRNn, RRAn, PosN, PosA, RRNe, RRAe, MSZoning_C (all), MSZoning_FV, MSZoning_RH, MSZoning_RL, MSZoning_RM, Street_Grvl, Street_Pave, Alley_Grvl, Alley_Pave, LotShape_IR1, LotShape_IR2, LotShape_IR3, LotShape_Reg, LandContour_Bnk, LandContour_HLS, LandContour_Low, LandContour_Lvl, Utilities_AllPub, LotConfig_Corner, LotConfig_CulDSac, LotConfig_FR2, LotConfig_FR3, LotConfig_Inside, LandSlope_Gtl, LandSlope_Mod, LandSlope_Sev, Neighborhood_Blmngtn, Neighborhood_Blueste, Neighborhood_BrDale, Neighborhood_BrkSide, Neighborhood_ClearCr, Neighborhood_CollgCr, Neighborhood_Crawfor, Neighborhood_Edwards, Neighborhoo

fillna: 
mean is best for columns where the data is mostly symmetric
median is best for when there are outliers which may skew your mean\
mode is best for when there is a certain predominant value in the column

use this to consider when to use which for fillna above, its 12 now so ill default fillna with mode and mean for now and adjust the column fillna methods more in detail in a later version.

# Making both training and test datasets conform to each other

In [4]:
SetX = set(X.columns)
SetTestX = set(X_test_fr.columns)
ToRemove = list(SetTestX - SetX)
# print(X.columns)
# print(X_test_fr.columns)

# ToRemove.append('Id')
ToAdd = list(SetX - SetTestX)
X_test_fr = X_test_fr.drop(ToRemove, axis=1)

# # Here we add columns present in traindata but absent in testdata to the testdata, X_test_fr
# ToAdd_df = pd.DataFrame(False, index=X.index, columns=ToAdd)
# X_test_fr = pd.concat([X_test_fr, ToAdd_df], axis=1)

# Instead of adding toadd to testdata, lets try removing toadd from traindata
X = X.drop(ToAdd, axis=1)

print(X.columns)
print(X_test_fr.columns)
print(TestIDs)


Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'MasVnrArea', 'BsmtQual', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       ...
       'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family',
       'SaleCondition_Normal', 'SaleCondition_Partial', 'TotalSF', 'TotalBth',
       'Age', 'GAge', 'ReAge'],
      dtype='object', length=260)
Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'MasVnrArea', 'BsmtQual', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       ...
       'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family',
       'SaleCondition_Normal', 'SaleCondition_Partial', 'TotalSF', 'TotalBth',
       'Age', 'GAge', 'ReAge'],
      dtype='object', length=260)
0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64


# Model Fitting: RandomForestClassifier

In [5]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from sklearn.tree import DecisionTreeRegressor

import xgboost as xgb
# model = RandomForestClassifier(random_state=0, bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100)

model = RandomForestRegressor(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
model.fit(X_train, y_train)
# checking feature importances
fi = pd.DataFrame(
    {
        'feature_names': X.columns, 
        'feature_importances': model.feature_importances_
    }
)
print(fi.sort_values(by='feature_importances', ascending=False).head(20))
pred = model.predict(X_test)
# gonna stop using accuracy_score and start using R^2, MAE, and MSE instead
r2 = r2_score(y_test, pred)
print(r2)
# 0.003424657534246575 accuracyscore
# 0.8512528653399029 r2score

    feature_names  feature_importances
3     OverallQual             0.548168
255       TotalSF             0.092730
12      GrLivArea             0.041249
10    TotalBsmtSF             0.035971
7      BsmtFinSF1             0.031936
5      MasVnrArea             0.027243
20     GarageArea             0.026664
19     GarageCars             0.019329
256      TotalBth             0.017088
257           Age             0.016972
2         LotArea             0.012570
6        BsmtQual             0.010361
1     LotFrontage             0.007490
9       BsmtUnfSF             0.006851
258          GAge             0.006722
259         ReAge             0.006455
21     WoodDeckSF             0.005617
4     OverallCond             0.004791
22    OpenPorchSF             0.004615
17   TotRmsAbvGrd             0.004114
0.8442868907431306


# Submission

In [6]:
model.fit(X, y)
prediction = model.predict(X_test_fr)

# Submission
output = pd.DataFrame({'Id': TestIDs, 'SalePrice': prediction})
print(output)
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

        Id  SalePrice
0     1461  124165.37
1     1462  150695.82
2     1463  176967.17
3     1464  183123.94
4     1465  199761.30
...    ...        ...
1454  2915   87465.00
1455  2916   89408.24
1456  2917  153586.36
1457  2918  117496.75
1458  2919  223390.90

[1459 rows x 2 columns]
Your submission was successfully saved!
